In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
import seaborn as sns
import math
from scipy import stats
import cooler
import cooltools.lib.plotting
from cooltools import insulation

import itertools

import os
import sys

# local imports 
import filters
import pore_c_utils as pcu

In [ ]:
dirpath = "/nfs/turbo/umms-indikar/shared/tools/Pore-C-Snakemake/results_ES5to12/matrix/"
cell = "run07"

filepath = f"{dirpath}NlaIII_{cell}_GRCm39_unphased.matrix.mcool"
print(filepath)

In [ ]:
cooler.fileops.list_coolers(filepath)

# File Load

In [ ]:
# resolution = 1000000 # 1MB
resolution = 100000 # 1 kb

clr = cooler.Cooler(f'{filepath}::resolutions/{resolution}')

In [ ]:
### to make a list of chromosome start/ends in bins:
chromstarts = []
for i in clr.chromnames:
    print(f'{i} : {clr.extent(i)}')
    chromstarts.append(clr.extent(i)[0])

In [ ]:
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

In [ ]:
assembly = pcu.loadAssembly("GRCm39_Assembly.txt")
assembly

In [ ]:
chromMap = dict(zip(assembly['RefSeq accession'], assembly['Chromosome']))
chromMap

In [ ]:
def plotHicMatrixChromosome(clr, chromosome, log=False, balance=False):
    
    A = clr.matrix(balance=balance).fetch(chromosome)
    if log:
        A = np.ma.log(A).filled(0) # log scale

    im = plt.matshow(
        A,
        extent=(0,clr.chromsizes[chromosome], clr.chromsizes[chromosome], 0),
        cmap='Reds'
    );

    if log:
        plt.colorbar(im, fraction=0.046, pad=0.04, label='Counts (log)');
    elif balance:
        plt.colorbar(im, fraction=0.046, pad=0.04, label='Counts (normalized)');
    else:
        plt.colorbar(im, fraction=0.046, pad=0.04, label='Counts');
        
    ax = plt.gca()
    format_ticks(ax)


    
chrom = 'NC_000068.8'
    
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 8, 8

plotHicMatrixChromosome(clr, chrom, log=True, balance=False)
titleString = f"Chromosome {chromMap[chrom]}"
plt.title(titleString)
plt.ylabel('Position (Mb)')


In [ ]:
# break

# KR Balancing

In [ ]:
bias, stats = cooler.balance_cooler(clr, rescale_marginals=True)

clr.bins()[:]['weight'] = bias
clr.bins()[:3]

In [ ]:
chrom = 'NC_000068.8'
    
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 8, 8

plotHicMatrixChromosome(clr, chrom, log=False, balance=True)
titleString = f"Chromosome {chromMap[chrom]}"
plt.title(titleString)
plt.ylabel('Position (Mb)')

In [ ]:


def plotCoverageChromosome(clr, chrom, log=True):
    cis_coverage, tot_coverage = cooltools.coverage(clr)
    extent = clr.extent(chrom)
    
    if log:
        cis_coverage = np.ma.log(cis_coverage)
        tot_coverage = np.ma.log(tot_coverage)
    
        plt.plot(cis_coverage[extent[0]:extent[1]], label='cis Coverage (log)')
        plt.plot(tot_coverage[extent[0]:extent[1]], label='Total Coverage (log)')
    else:        
        plt.plot(cis_coverage[extent[0]:extent[1]], label='cis Coverage')
        plt.plot(tot_coverage[extent[0]:extent[1]], label='Total Coverage')
    plt.legend()
    
    
    
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 8, 4
plotCoverageChromosome(clr, chrom)
titleString = f"Chromosome {chromMap[chrom]} Coverage"
plt.title(titleString)
plt.ylabel('Coverage (log)')
plt.xlabel('Position (Mb)')


In [ ]:
cis_coverage, tot_coverage = cooltools.coverage(clr)
extent = clr.extent(chrom)

ratio = cis_coverage[extent[0]:extent[1]] / tot_coverage[extent[0]:extent[1]] 
plt.bar(range(len(ratio)), ratio)
plt.title('Cis/Total Ratio')
plt.ylabel('Cis Contacts / Total Contacts')
plt.xlabel('Position (Mb)')

In [ ]:
np.nanmean(ratio)

# Smoothing

In [ ]:


# cg = cooltools.lib.numutils.adaptive_coarsegrain(clr.matrix(balance=True).fetch(chrom),
#                                                  clr.matrix(balance=False).fetch(chrom),
#                                                  cutoff=10, 
#                                                  max_levels=20)

# cgi = cooltools.lib.numutils.interp_nan(cg)

# im = plt.matshow(
#     cgi,
#     extent=(0,clr.chromsizes[chrom], clr.chromsizes[chrom], 0),
#     cmap='Reds'
# );


# ax = plt.gca()
# format_ticks(ax)


# Insulation

In [ ]:
import cooltools.lib.plotting
from cooltools import insulation

from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import bioframe

In [ ]:
resolution = 100000

clr = cooler.Cooler(f'{filepath}::resolutions/{resolution}')


# balance the matrix
bias, stats = cooler.balance_cooler(clr, rescale_marginals=True)
clr.bins()[:]['weight'] = bias

In [ ]:
windows = [3*resolution, 5*resolution, 10*resolution, 25*resolution]
insulation_table = insulation(clr, windows, verbose=False)
insulation_table.head()

In [ ]:
def rotate45deg(data, resolution=1):
    start_pos_vector = [0+resolution*i for i in range(len(data)+1)]
    n = data.shape[0]
    t = np.array([[1, 0.5], [-1, 0.5]])
    A = np.dot(np.array([(i[1], i[0])
                         for i in itertools.product(start_pos_vector[::-1],
                                                           start_pos_vector)]), t)
    x = A[:, 1].reshape(n + 1, n + 1)
    y = A[:, 0].reshape(n + 1, n + 1)
    
    return x, y

In [ ]:
# Functions to help with plotting
def pcolormesh_45deg(ax, matrix_c, start=0, resolution=1, *args, **kwargs):
    start_pos_vector = [start+resolution*i for i in range(len(matrix_c)+1)]
    n = matrix_c.shape[0]
    t = np.array([[1, 0.5], [-1, 0.5]])
    matrix_a = np.dot(np.array([(i[1], i[0])
                                for i in itertools.product(start_pos_vector[::-1],
                                                           start_pos_vector)]), t)
    x = matrix_a[:, 1].reshape(n + 1, n + 1)
    y = matrix_a[:, 0].reshape(n + 1, n + 1)
    im = ax.pcolormesh(x, y, np.flipud(matrix_c), *args, **kwargs)
#     im.set_rasterized(True)
    return im

In [ ]:
insulation_table.columns

In [ ]:
start = clr.extent(chrom)[0]
end = clr.extent(chrom)[1]
region = (chrom, start, end)
ylim = 300 # in base pairs
insultionValue = windows[1]
insulationColumn = f'log2_insulation_score_{insultionValue}'
boundaryColumn = f'is_boundary_{insultionValue}'


# get region
insul_region = bioframe.select(insulation_table, chrom)
insul_region['midpoint'] = insul_region[['start', 'end']].mean(axis=1)
isBoundary = insul_region[insul_region[boundaryColumn] == True]


norm = LogNorm(vmax=100, vmin=0.0001)
data = clr.matrix(balance=True).fetch(chrom)


f, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5))


x, y = rotate45deg(data, resolution=1)
ax1.pcolormesh(x, y, np.flipud(data), cmap='blues',  norm=norm)

ax1.set_ylim(0, ylim)


for bound in isBoundary['midpoint']:
    binBound = bound / resolution
    ax1.axvline(x=binBound, lw=0.5, ls="--", c='r')

format_ticks(ax1, rotate=False)


# # plot insulation scores below
ax2.plot(insul_region['start'],
         insul_region[insulationColumn], 
         lw=0.5)


ylimMin = np.nanpercentile(insul_region[insulationColumn], 0.5)
# ylimMin = insul_region[insulationColumn].min()
ax2.set_ylim(ylimMin, insul_region[insulationColumn].max())
ax2.set_title(insulationColumn)

# add boundary calling
isBoundary = insul_region[insul_region[boundaryColumn] == True]

ax2.scatter(isBoundary['start'],
            isBoundary[insulationColumn], 
            c='r',
            s=5)
    

format_ticks(ax2, y=False, rotate=False)

plt.subplots_adjust(hspace = 0.75)


In [ ]:
def loadTADBoundary(filepath):
    df = pd.read_csv(filepath, sep='\t', header=None)
    df.columns = ['phase', 'start', 'end', 'chrom']
    df['phase'] = df['phase'].apply(lambda x: x.split("_")[0])
    return df

def mergeAssembly(tad, assembly):
    
    tad['chrom'] = tad['chrom'].astype('str')
    tad = pd.merge(left=tad, 
                   right=assembly, 
                   left_on='chrom', 
                   right_on='Chromosome')
    
    return tad



tad = loadTADBoundary('F121_mESC_TADS.txt')
tad = mergeAssembly(tad, assembly)

# filter to chrom
tad = tad[tad['RefSeq accession'] == chrom]

# filter to phase
phase = 'G1'
tad = tad[tad['phase'] == phase]

# add an approximate bin
tad['binStart'] = np.ceil(tad['start'] / resolution).astype(int)
tad['binEnd'] = np.ceil(tad['end'] / resolution).astype(int)

tad.head()

In [ ]:
alpha = 0.7

labels = []
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 8, 3

for i, window in enumerate(windows):
    windowColumns = [x for x in insul_region.columns if str(window) in x]
    
    boundaryColumn = [x for x in windowColumns if 'is_boundary' in x][0]
    
    isBoundary = insul_region[insul_region[boundaryColumn] == True]
    
    label = f'{int(window/resolution)} Mb Window'
    labels.append(label)
    plt.scatter(x=isBoundary['start'], 
                y=[i]*len(isBoundary), 
                alpha=alpha,
                s=20,
                c=f"C{i+1}",
#                 marker="|",
                marker=".",
                label = label)


plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
ax = plt.gca()
ax.set_ylim(-0.5, 4)

for idx, row in tad.iterrows():
    plt.axvline(x=row['start'], lw=1, ls='--', alpha=0.3)

format_ticks(ax, y=False, rotate=False)

y = list(range(len(windows)))
plt.yticks(y, labels)
    

In [ ]:
insul_region.columns

In [ ]:
clr.bins()[10000]

In [ ]:
tad['start'].max()

In [ ]:
clr.binsize